# Project notebook

In [1]:
import numpy as np
import pandas as pd
import neo4j
import os
import psycopg2

## Neo4j setup

In [2]:
driver = neo4j.GraphDatabase.driver(uri="neo4j://neo4j:7687", auth=("neo4j","ucb_mids_w205"))

In [3]:
session = driver.session(database="neo4j")

In [4]:
def my_neo4j_wipe_out_database():
    "wipe out database by deleting all nodes and relationships"
    
    query = "match (node)-[relationship]->() delete node, relationship"
    session.run(query)
    
    query = "match (node) delete node"
    session.run(query)

In [5]:
def my_neo4j_create_node(country):
    "create a node with label Country"
    
    query = """
    
    CREATE (:Country {name: $country})
    
    """
    
    session.run(query, country=country)
    

In [6]:
def my_neo4j_create_relationship_one_way(from_country, to_country, weight):
    "create relationships one way between two countries with a weight"
    
    query = """
    
    MATCH (from:Country), 
          (to:Country)
    WHERE from.name = $from_country and to.name = $to_country
    CREATE (from)-[:LINK {weight: $weight}]->(to)

    
    """
    
    session.run(query, from_country=from_country, to_country=to_country, weight=weight)
    

In [7]:
my_neo4j_wipe_out_database()

## Postgres setup

In [8]:
connection = psycopg2.connect(
    user = "postgres",
    password = "ucb",
    host = "postgres",
    port = "5432",
    database = "postgres"
)

In [9]:
cursor = connection.cursor()

## Importing refugee movement data

In [10]:
# Load population.csv from Github
url = "https://raw.githubusercontent.com/rfordatascience/tidytuesday/master/data/2023/2023-08-22/population.csv"
df = pd.read_csv(url)
print(df.shape)

df.head()

(64809, 16)


,year,coo_name,coo,coo_iso,coa_name,coa,coa_iso,refugees,asylum_seekers,returned_refugees,idps,returned_idps,stateless,ooc,oip,hst
0,2010,Afghanistan,AFG,AFG,Afghanistan,AFG,AFG,0,0,0,351907,3366,0,838250,NaN,NaN
1,2010,Iran (Islamic Rep. of),IRN,IRN,Afghanistan,AFG,AFG,30,21,0,0,0,0,0,NaN,NaN
2,2010,Iraq,IRQ,IRQ,Afghanistan,AFG,AFG,6,0,0,0,0,0,0,NaN,NaN
3,2010,Pakistan,PAK,PAK,Afghanistan,AFG,AFG,6398,9,0,0,0,0,0,NaN,NaN
4,2010,Egypt,ARE,EGY,Albania,ALB,ALB,5,0,0,0,0,0,0,NaN,NaN


## Split Data into Individual Years
- ~~Attempt with 2 groups: (2010-2016, 2017-2022)~~
- **Attempt with 3 single years: 2014, 2018, 2022 - look at single year graphs, each 6 years apart to see trends**

In [11]:
# Create dfs for specific years
df_2014 = df[df['year'] == 2014]
df_2018 = df[df['year'] == 2018]
df_2022 = df[df['year'] == 2022]

# Validate that the DataFrames only contain the expected year
assert set(df_2014['year'].unique()) == {2014}
assert set(df_2018['year'].unique()) == {2018}
assert set(df_2022['year'].unique()) == {2022}

In [12]:
print(df_2014.shape)
print(df_2018.shape)
print(df_2022.shape)

(4746, 16)
(5264, 16)
(5930, 16)


## Preprocess refugee movement data to map countries to their country code (ex. Afghanistan -> AFG, Iran (Islamic Rep. of) -> IRN): for 2014, 2018, 2022

In [ ]:
def preprocess_country_nodes_to_csv(df: pd.DataFrame, output_file_path: str) -> None:
    """Map countries to their country code from a DataFrame containing origin and asylum data.
    Saves the resulting country nodes table to CSV file specified by output_file_path.

    Args:
        df (pd.DataFrame): Refugee data. Will contain the split data for individual years (2014, 2018, 2022).
        output_file_path (str): The name of the file path to save the country nodes table to.
    Returns:
        None.
    """
    # Create origin and asylum countries DF for appending
    origin_df = df[['coo_name', 'coo']].rename(columns = {'coo_name': 'country_full', 'coo': 'country'})
    asylum_df = df[['coa_name', 'coa']].rename(columns = {'coa_name': 'country_full', 'coa': 'country'})
    nodes_df = pd.concat([origin_df, asylum_df])

    # Create countries table (nodes)
    nodes_df = nodes_df.drop_duplicates()

    # Ensure 1:1 mapping between country names and codes
    print("Nodes DF unique? ", nodes_df['country_full'].is_unique)

    # Specify file paths for saving CSVs
    main_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))
    nodes_path = os.path.join(main_dir, output_file_path)
    nodes_df.to_csv(nodes_path, index = False)
    return

In [ ]:
preprocess_country_nodes_to_csv(df_2014, 'nodes_2014.csv')
preprocess_country_nodes_to_csv(df_2018, 'nodes_2018.csv')
preprocess_country_nodes_to_csv(df_2022, 'nodes_2022.csv')

## Load CSV file containing country nodes data into postgres

In [ ]:
def load_country_nodes_csv_into_postgres(output_file_name: str) -> list[tuple[str]]:
    """
    Creates a "countries" table in Postgres, loads data from the specified CSV file into the table,
    and then returns the data using `cursor.fetchall()`.
    
    Args:
        output_file_name (str): The name of the CSV file containing the country nodes data 
            to be loaded into the Postgres table.
    Returns:
        list[tuple[any]]: A list of 1-item tuples where each tuple represents a row containing 
            the country code from the country table.
    """
    # Query into countries table for nodes
    connection.rollback()

    query = """

    drop table if exists countries;

    create table countries (
      country_full varchar(100),
      country varchar(100)
    );

    copy countries (country_full, country)
    from %(file_path)s delimiter ',' NULL '' csv header;

    select country
    from countries
    order by country;

    """
    
    file_path = f"/user/projects/project-3-dliang5299/{output_file_name}"
    cursor.execute(query, {'file_path': file_path})
    
    connection.rollback()

    rows = cursor.fetchall()
    return rows

## Creating + visualizing country nodes from postgres into Neo4j for 2014, 2018, 2022

In [ ]:
my_neo4j_wipe_out_database()
# Load country data from 2014 into postgres and create neo4j nodes
rows = load_country_nodes_csv_into_postgres('nodes_2014.csv')
for row in rows:
    country = row[0]
    my_neo4j_create_node(country)

In [ ]:
my_neo4j_wipe_out_database()
# Load country data from 2018 into postgres and create neo4j nodes
rows = load_country_nodes_csv_into_postgres('nodes_2018.csv')
for row in rows:
    country = row[0]
    my_neo4j_create_node(country)

In [ ]:
my_neo4j_wipe_out_database()
# Load country data from 2022 into postgres and create neo4j nodes
rows = load_country_nodes_csv_into_postgres('nodes_2022.csv')
for row in rows:
    country = row[0]
    my_neo4j_create_node(country)

## Create migration movement relationships for 2014, 2018, 2022

In [ ]:
def create_migration_movement_relationship_to_csv(df: pd.DataFrame, output_file_path: str) -> None:
    """Create migration movement relationships for countries paired via origin and asylum.
    Saves the resulting migration relationships table to CSV file specified by output_file_path.

    Args:
        df (pd.DataFrame): Refugee data. Will contain the split data for individual years (2014, 2018, 2022).
        output_file_path (str): The name of the file path to save the country nodes table to.
    Returns:
        None.
    """

    # Aggregate migration across all years by country pairs
    migration_df = df.groupby(['coo', 'coa'])['refugees'].sum().reset_index()
    migration_df = migration_df.rename(columns = {'coo': 'from_country', 'coa': 'to_country'})
    migration_df = migration_df[migration_df['refugees'] > 0]

    # Specify file paths for saving CSV
    main_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))
    migration_path = os.path.join(main_dir, output_file_path)
    migration_df.to_csv(migration_path, index = False)
    

    # Display summary stats for refugees
    migration_df.describe()

In [ ]:
create_migration_movement_relationship_to_csv(df_2014, 'migration_2014.csv')
create_migration_movement_relationship_to_csv(df_2018, 'migration_2018.csv')
create_migration_movement_relationship_to_csv(df_2022, 'migration_2022.csv')

## Load CSV file containing migration data into postgres

In [ ]:
def load_migration_csv_into_postgres(output_file_name: str) -> list[tuple[any]]:
    """
    Creates a "migrations" table in Postgres, loads data from the specified CSV file into the table,
    and then returns the data using `cursor.fetchall()`.
    
    Args:
        output_file_name (str): The name of the CSV file containing the migration data to be loaded into the Postgres table.
    Returns:
        list[tuple[any]]: A list of tuples where each tuple represents a row in the migration table.
    """
    # Query into migration table for relationships
    connection.rollback()
    
    query = """

    drop table if exists migration;

    create table migration (
      to_country varchar(100),
      from_country varchar(100),
      refugees numeric(8)
    );

    copy migration (to_country, from_country, refugees)
    from %(file_path)s delimiter ',' NULL '' csv header;

    select to_country, from_country, refugees
    from migration
    order by to_country, from_country;

    """
    
    file_path = f"/user/projects/project-3-dliang5299/{output_file_name}"
    cursor.execute(query, {'file_path': file_path})
    
    connection.rollback()

    rows = cursor.fetchall()
    return rows

## Creating migration movement graphs from postgres into Neo4j (Only run ONE of 2014, 2018, and 2022 cells based on the year you are assigned, DO NOT run all 3)

In [ ]:
## Load country data from 2014 into postgres and create neo4j nodes
rows = load_migration_csv_into_postgres('migration_2014.csv')

for row in rows:
    
    from_country = row[0]
    to_country = row[1]
    weight = row[2]
    
    my_neo4j_create_relationship_one_way(from_country, to_country, int(weight))


In [ ]:
# rows[:5] # list first few rows in list 

In [ ]:
## Load country data from 2018 into postgres and create neo4j nodes
rows = load_migration_csv_into_postgres('migration_2018.csv')

for row in rows:
    
    from_country = row[0]
    to_country = row[1]
    weight = row[2]
    
    my_neo4j_create_relationship_one_way(from_country, to_country, int(weight))


In [ ]:
## Load country data from 2022 into postgres and create neo4j nodes
rows = load_migration_csv_into_postgres('migration_2022.csv')

for row in rows:
    
    from_country = row[0]
    to_country = row[1]
    weight = row[2]
    
    my_neo4j_create_relationship_one_way(from_country, to_country, int(weight))


## Explore relationships + algorithms for 20XX

In [ ]:
def my_neo4j_run_query_pandas(query, **kwargs):
    "run a query and return the results in a pandas dataframe"
    
    result = session.run(query, **kwargs)
    
    df = pd.DataFrame([r.values() for r in result], columns=result.keys())
    
    return df

In [ ]:
def my_neo4j_nodes_relationships():
    "print all the nodes and relationships"
   
    print("-------------------------")
    print("  Nodes:")
    print("-------------------------")
    
    query = """
        match (n) 
        return n.name as node_name, labels(n) as labels
        order by n.name
    """
    
    df = my_neo4j_run_query_pandas(query)
    
    number_nodes = df.shape[0]
    
    display(df)
    
    print("-------------------------")
    print("  Relationships:")
    print("-------------------------")
    
    query = """
        match (n1)-[r]->(n2) 
        return n1.name as node_name_1, labels(n1) as node_1_labels, 
            type(r) as relationship_type, n2.name as node_name_2, labels(n2) as node_2_labels
        order by node_name_1, node_name_2
    """
    
    df = my_neo4j_run_query_pandas(query)
    
    number_relationships = df.shape[0]
    
    display(df)
    
    density = (2 * number_relationships) / (number_nodes * (number_nodes - 1))
    
    print("-------------------------")
    print("  Density:", f'{density:.1f}')
    print("-------------------------")
    

In [ ]:
# for 20XX database loaded 
my_neo4j_nodes_relationships()

# observe a density of .2 --> sparse graph or low density

### Alg #1: Degree Centrality 

High degree centrality shows countries with most creation of refugees / grant asylum 
In this graph, we see top countries sending refugees: 

In [ ]:
# degree centrality 
query = "CALL gds.graph.drop('ds_graph', false) yield graphName"
session.run(query)

query = "CALL gds.graph.project('ds_graph', 'Country', 'LINK')"
session.run(query)

In [ ]:
query = """

CALL gds.degree.stream('ds_graph')
YIELD nodeId, score
RETURN gds.util.asNode(nodeId).name AS name, score as degree
ORDER BY degree DESC, name
limit 20

"""

my_neo4j_run_query_pandas(query)

This shows top countries sending refugees: 
- Somalia
- Democratic Republic of the Congo
- Iraq
- Sudan
- Iran
- Ethiopia
- Afghanistan
- Liberia 

## Alg #2: Pagerank

Most influential countries granting asylum 

In [ ]:
query = """

CALL gds.pageRank.stream('ds_graph',
                         { maxIterations: $max_iterations,
                           dampingFactor: $damping_factor}
                         )
YIELD nodeId, score
RETURN gds.util.asNode(nodeId).name AS name, score as page_rank
ORDER BY page_rank DESC, name ASC
limit 20

"""

max_iterations = 20
damping_factor = 0.05

my_neo4j_run_query_pandas(query, max_iterations=max_iterations, damping_factor=damping_factor)


## Alg #3: Randomized-Approximate Brandes 
Since betweenness can be very time consuming and expensive due to all pairs shortest path

High betweeness shows the intermediate countries that act as places of temporary refuge 

In [ ]:
query = """

CALL gds.betweenness.stream('ds_graph', {samplingSize: $sampling_size, samplingSeed: $sampling_seed})
YIELD nodeId, score
RETURN gds.util.asNode(nodeId).name AS name, score as betweenness
ORDER BY betweenness DESC
limit 20

"""

sampling_size = 5
sampling_seed = 0

my_neo4j_run_query_pandas(query, sampling_size=sampling_size, sampling_seed=sampling_seed)
